# Line Solver Algorithm


In [5]:
import re
from itertools import product

In [6]:
def clue_to_regex(clue):
    """
    Convert the clue to a regular expression pattern.
    Each block in the clue must have '1' repeated block_size times
    and separated by at least one '0', except at boundaries.
    """
    parts = [f"1{{{c}}}" for c in clue]  # Convert each block into a '1' repeated `c` times
    # Join the parts with '0+' to allow for flexible spacing between blocks
    pattern = "0+".join(parts)
    # Allow leading and trailing zeros
    return f"^0*{pattern}0*$"

def generate_line_domains(length, clue):
    """
    Generate all valid binary sequences of a given length based on a row or column clue.
    Uses a regular expression to filter out invalid patterns.
    """
    # Convert the clue to a regular expression
    regex = clue_to_regex(clue)
    # Generate all possible binary sequences of the given length
    all_sequences = [''.join(seq) for seq in product("01", repeat=length)]
    # Filter sequences that match the regex
    valid_sequences = [list(map(int, seq)) for seq in all_sequences if re.fullmatch(regex, seq)]
    return valid_sequences

In [7]:
def choose_line_heuristic(row_domains, col_domains, assigned, stalled_lines):
    min_domain_size = float('inf')
    max_unsolved_cells = -1
    chosen_line_type = None
    chosen_index = None

    # Check rows
    for idx, domain in enumerate(row_domains):
        line_id = f"row_{idx}"
        if line_id not in assigned and line_id not in stalled_lines:
            unsolved_cells = sum(1 for valid_row in domain if len(valid_row) > 1)
            if len(domain) < min_domain_size or (
                len(domain) == min_domain_size and unsolved_cells > max_unsolved_cells
            ):
                min_domain_size = len(domain)
                max_unsolved_cells = unsolved_cells
                chosen_line_type = "row"
                chosen_index = idx

    # Check columns
    for idx, domain in enumerate(col_domains):
        line_id = f"col_{idx}"
        if line_id not in assigned and line_id not in stalled_lines:
            unsolved_cells = sum(1 for valid_col in domain if len(valid_col) > 1)
            if len(domain) < min_domain_size or (
                len(domain) == min_domain_size and unsolved_cells > max_unsolved_cells
            ):
                min_domain_size = len(domain)
                max_unsolved_cells = unsolved_cells
                chosen_line_type = "col"
                chosen_index = idx

    # If no line is found, return None to signal no viable options
    if chosen_line_type is None:
        return None, None, None

    return chosen_index, (
        row_domains[chosen_index] if chosen_line_type == "row" else col_domains[chosen_index]
    ), chosen_line_type



In [8]:
def solve_nonogram_no_backtracking(grid_size, row_clues, col_clues):
    """
    Solve the nonogram puzzle based on row and column clues without backtracking.
    If no unique solution is found, report it.
    """
    num_rows, num_cols = grid_size
    grid = [[set([0, 1]) for _ in range(num_cols)] for _ in range(num_rows)]
    
    # Precompute valid row and column domains
    row_domains = [generate_line_domains(num_cols, clue) for clue in row_clues]
    col_domains = [generate_line_domains(num_rows, clue) for clue in col_clues]
    
    # Enforce arc consistency
    while enforce_arc_consistency(grid, row_domains, col_domains):
        pass
    
    # Check if the grid has a unique solution
    unsolved_cells = any(len(cell) > 1 for row in grid for cell in row)
    if unsolved_cells:
        print("No unique solution found.")
        return None  # Return None to indicate no solution was found

    # Convert the grid to the final solved grid (if possible)
    solved_grid = []
    for row in grid:
        solved_grid.append([list(cell)[0] for cell in row])  # Take the single value from each set
    
    return solved_grid


def enforce_arc_consistency(grid, row_domains, col_domains):
    assigned = set()  # Track lines already processed
    stalled_lines = set()  # Track lines that didn't lead to progress
    revised = False
    while True:
        chosen_index, chosen_domain, chosen_line_type = choose_line_heuristic(
            row_domains, col_domains, assigned, stalled_lines
        )
        
        if chosen_index is None:  # No more lines to process
            break

        if chosen_line_type == "row":
            row_idx = chosen_index
            for col_idx, cell in enumerate(grid[row_idx]):
                valid_values = {valid_row[col_idx] for valid_row in chosen_domain}
                original_domain = cell.copy()
                cell.intersection_update(valid_values)

                if cell != original_domain:
                    revised = True
                    stalled_lines.clear()  # Clear stalled lines since progress was made
                    # Update column domains
                    col_domains[col_idx] = [
                        valid_col for valid_col in col_domains[col_idx]
                        if valid_col[row_idx] in cell
                    ]
            assigned.add(f"row_{row_idx}")

        else:  # chosen_line_type == "col"
            col_idx = chosen_index
            for row_idx in range(len(grid)):
                cell = grid[row_idx][col_idx]
                valid_values = {valid_col[row_idx] for valid_col in chosen_domain}
                original_domain = cell.copy()
                cell.intersection_update(valid_values)

                if cell != original_domain:
                    revised = True
                    stalled_lines.clear()  # Clear stalled lines since progress was made
                    # Update row domains
                    row_domains[row_idx] = [
                        valid_row for valid_row in row_domains[row_idx]
                        if valid_row[col_idx] in cell
                    ]
            assigned.add(f"col_{col_idx}")

        # If no progress was made, mark the line as stalled
        if not revised:
            stalled_lines.add(f"{chosen_line_type}_{chosen_index}")

    return revised



def draw(solution):
    if solution:    
        for row in solution:
            print(''.join('#' if cell == 1 else '.' for cell in row))
            
# Example Usage
# row_clues = [[1], [2], [5], [2], [1]]
# 
# col_clues = [[1], [2, 1], [3], [1, 2], [1]]
# 
# solution = solve_nonogram_no_backtracking((5, 5), row_clues, col_clues)

# #works perfectly
# 
# row_clues = [[2], [4], [6], [2], [8], [2], [10], [2], [2], [2]]
# col_clues = [[1], [1,1], [1,1,1], [2,1,1], [10], [10], [2,1,1], [1,1,1], [1,1], [1]]
# 
# solution = solve_nonogram_no_backtracking((10, 10), row_clues, col_clues)


row_clues = [[3], [2,2], [5], [5], [3], [1], [1], [2], [1], [2]]
col_clues = [[3], [5], [1,8], [5,1,1], [3]]  
solution = solve_nonogram_no_backtracking((10, 5), row_clues, col_clues)

 #works perfectly

draw(solution)

.###.
##.##
#####
#####
.###.
..#..
..#..
..##.
..#..
..##.


Trying line solver algorithm on Ani's examples

In [9]:
row_clues = [[4], [1, 1], [1, 1], [4, 4], [1, 1,1, 1],
        [1, 2, 1, 2], [1, 1, 1, 1], [4, 4, 1], [1, 1, 1], [1, 1, 1],
        [1, 1, 1, 2], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1],
                    [1, 1, 1, 1], [1, 2, 1], [1, 1], [1, 1], [6]]

col_clues = [[5], [1, 1], [3, 1, 12], [1, 5, 1], [1, 6, 1], [1, 1, 1],
        [1, 5, 1, 1], [3, 9, 1], [1, 1, 1, 1], [5, 9], [2], [2]]

solution = solve_nonogram_no_backtracking((20, 12), row_clues, col_clues)

for row in solution:
    print(''.join('#' if cell == 1 else '.' for cell in row))

...####.....
..#....#....
..#....#....
####..####..
#..#..#..#..
#.##..#.##..
#..#..#..#..
####..####.#
..#....#...#
..#....#..#.
..#.#..#.##.
..#.#..#.#..
..#.#..#.#..
..#.#..#.#..
..#.#..#.#..
..#.#..#.#..
..#..##..#..
..#......#..
..#......#..
...######...


In [7]:
# THANKS
# row_clues = [[0], [3,1,1,3,1,1,1,1,3], [1,1,1,1,1,1,1,1,1,1], [1,3,3,2,1,2,3], [1,1,1,1,1,1,2,1,1,1],
#         [1,1,1,1,1,1,1,1,1,3], [0]]
# 
# col_clues = [[1], [5], [1], [0], [5],
#         [1], [5],[0], [5], [1, 1], [5], [0], [5], [1], [1], [5], [0], [5],
#         [1], [2,2], [0], [3, 1], [1, 1, 1], [1,3]]

# solution = solve_nonogram_no_backtracking((7, 24), row_clues, col_clues)
# 
# for row in solution:
#     print(''.join('#' if cell == 1 else '.' for cell in row))

In [37]:
row_clues = [[3, 2], [6], [1, 1]]

col_clues = [[1], [3], [2], [3], [1], [2], [1]]

solution = solve_nonogram_no_backtracking((3, 7), row_clues, col_clues)

draw(solution)

.###.##
######.
.#.#...


In [21]:
row_clues = [[1], [1]]

col_clues = [[1], [1]]

solution = solve_nonogram_no_backtracking((2,2), row_clues, col_clues)


No unique solution found.


In [38]:
row_clues = [[3], [2,2],[1,7], [3,2], [8], [3,1], [1,1,2], [1,1,1,2], [1,1,2,1], [4,2]  ]

col_clues = [[3,3], [2,2,1], [1,3,1,1], [3,1,2], [3,1,1], [1,1,2], [1,1,2], [3,2], [6,1], [2,2]  ]

solution = solve_nonogram_no_backtracking((10,10), row_clues, col_clues)
draw(solution)

###.......
##......##
#..#######
..###..##.
.########.
###.....#.
#..#...##.
#.#.#.##..
.#.#.##..#
..####..##


In [39]:
import time
row_clues = [
        [9], [1, 1, 1, 1, 1, 1], [2, 1, 1, 2], [1, 1, 1, 1, 1, 1, 1, 1], [1, 4, 4, 1],
        [1, 5, 5, 1], [1, 1, 3, 3, 1, 1], [1, 2, 2, 1], [1, 1, 1, 1, 1], [1, 3, 3, 1],
        [1, 11, 1], [1, 9, 1], [2, 7, 2], [1, 1, 1, 1, 1, 1], [9]
    ]

col_clues = [
        [9], [1, 1], [6, 2, 2], [1, 2, 4, 1], [2, 4, 6], [1, 4, 3, 1], [4, 3, 5], [1, 1, 3, 1],
        [4, 3, 5], [1, 4, 3, 1], [2, 4, 6], [1, 2, 4, 1], [6, 2, 2], [1, 1], [9]
    ]
m, n = len(row_clues), len(col_clues)
grid_size = (m, n)

start_time = time.time()
solution = solve_nonogram_no_backtracking(grid_size, row_clues, col_clues)
end_time = time.time()
print(end_time - start_time)

draw(solution)

0.3713710308074951
...#########...
..#.#.#.#.#.#..
.##...#.#...##.
#.#.#.#.#.#.#.#
#.####...####.#
#.#####.#####.#
#.#.###.###.#.#
#....##.##....#
#..#...#...#..#
#.###.....###.#
#.###########.#
#..#########..#
.##.#######.##.
..#.#.#.#.#.#..
...#########...


In [40]:
row_constraints = [
    [9], [1, 1, 1, 1, 1, 1], [2, 1, 1, 2], [1, 1, 1, 1, 1, 1, 1, 1], [1, 4, 4, 1],
    [1, 5, 5, 1], [1, 1, 3, 3, 1, 1], [1, 2, 2, 1], [1, 1, 1, 1, 1], [1, 3, 3, 1],
    [1, 11, 1], [1, 9, 1], [2, 7, 2], [1, 1, 1, 1, 1, 1], [9]
]

col_constraints = [
    [9], [1, 1], [6, 2, 2], [1, 2, 4, 1], [2, 4, 6], [1, 4, 3, 1], [4, 3, 5], [1, 1, 3, 1],
    [4, 3, 5], [1, 4, 3, 1], [2, 4, 6], [1, 2, 4, 1], [6, 2, 2], [1, 1], [9]
]
m, n = len(row_constraints), len(col_constraints)

start_time = time.time()
solve_nonogram_no_backtracking((m,n), row_constraints, col_constraints)
end_time = time.time()
print(end_time - start_time)


0.36136507987976074


In [41]:

row_constraints = [
    [5, 1], [2, 2, 1], [1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1], [2, 2, 1], [5, 1],
    [1], [1, 1, 1], [5, 1], [5, 1], [5, 1], [3, 1], [1, 2], [3]
]
col_constraints = [
    [5], [2, 2], [1, 1, 1], [1, 1, 3], [1, 1, 1, 5], [2, 2, 5], [5, 5, 1], [3, 1], [2], [14]
]

m, n = len(row_constraints), len(col_constraints)

In [42]:
def compute_time(row_constraints, col_constraints):
    m, n = len(row_constraints), len(col_constraints)
    start_time = time.time()
    solution = solve_nonogram_no_backtracking((m,n), row_constraints, col_constraints)
    end_time = time.time()
    draw(solution)
    return end_time - start_time

In [43]:
compute_time(row_constraints, col_constraints)

.#####...#
##...##..#
#.....#..#
#.#.#.#..#
#.....#..#
##...##..#
.#####...#
.........#
....#.#..#
...#####.#
...#####.#
...#####.#
....###..#
.....#..##
......###.


0.1383838653564453

74114

73862  (14x15)

In [81]:
row_constraints = [
    [8,3], [7,4,2], [3,1,4,1,1], [1,1,3,2,1], [1,7], [2,5], [4,2], [2,5,1,2], [1,3,5], [1,1,4,1], [1,2,5,1], [2,1,1,2,1], [3,1,3,2], [5,4]
]
col_constraints = [
    [2,7], [3,1,3], [4,2], [3,1,1], [2,1,2,1], [9], [2,4,1], [1,3,3,1], [4,2,2,1], 
    [5,5], [2,7,1], [1,4,4,1], [1,4,2,2,1], [2,2,3,2], [4,4]
]
compute_time(row_constraints, col_constraints)

########....###
#######.####.##
.###.#.####.#.#
..#..#.###.##.#
.....#.#######.
.....##..#####.
.....####.##...
##..#####.#.##.
#....###.#####.
#...#...####.#.
#..##...#####.#
##.....#.#.##.#
###...#.###..##
#####......####


0.2518641948699951

73770  16x12

In [82]:
row_constraints = [
    [3], [5], [3,2], [7], [5], [3], [3], [6], [5,2], [7,2], [3,2],
    [3,3], [7], [4], [1,1], [4]
]
col_constraints = [
    [1], [2], [3], [5], [2,3], [4,3,4], [6,3,2,1], [9,4], [2,5,2,1], [4,7], [2,5], [1]
]
compute_time(row_constraints, col_constraints)

......###...
.....#####..
.....###.##.
.....#######
.....#####..
......###...
.......###..
.....######.
...#####.##.
#######..##.
.###.....##.
..###...###.
...#######..
....####....
.....#.#....
.....####...


0.2991368770599365

73842 16x14

In [49]:
row_constraints = [
    [10], [12], [3,3], [2,1,2,1,2], [1,2,3], [3,5,2], [7], [3,4,1], [1,4], [2,5],
    [3,2], [2,1], [2,1], [4], [2], [2]
]
col_constraints = [
    [2], [3,1], [3,4], [2,1,5], [2,2,3], [2,1,2,4], [2,5,2,4],
    [2,6,3], [2,6,2], [2,2,5], [2,2,2], [3,2,1], [3,1], [2]
]
compute_time(row_constraints, col_constraints)

..##########..
.############.
###........###
##.#.##...#.##
..#...##.###..
.###.#####.##.
..#######.....
..###.####.#..
...#...####...
...##.#####...
....###.##....
....##...#....
.....##.#.....
.....####.....
......##......
......##......


0.44948601722717285

73788 17x17

In [83]:
row_constraints = [
    [7], [9], [9], [13], [12], [11], [6,1], [4,2,1], [2,1,2], [1,1,1], [1,2,3], [2,3,2,1], [1,5,2], [5,3], [5,3], [4,2], [2,2] 
]
col_constraints = [
    [1], [1], [2], [8], [6,1,1,4], [7,5], [7,3], [7,5], [6,5], 
    [6,1,3], [6,3,1], [6,2], [5,4], [6, 1, 3], [2,5], [1,4], [1,2]
]
compute_time(row_constraints, col_constraints)

.....#######.....
....#########....
....#########....
....#############
...############..
...###########...
..######.....#...
####.....##..#...
...##.....#.##...
...#......#.#....
...#....##..###..
...##..###.##.#..
.....#.#####.##..
....#####....###.
....#####....###.
....####.......##
....##.........##


1.583857774734497

72956 20x17

In [84]:
row_constraints = [
    [6,2,4], [3,2,1,1,4], [3,2,1,1,1], [4,1,2,2,2], [3,1,1,2], [1,1,1,1], [1,1,2,3], [1,4,1,2,1], [1,1,3,4], [1,2,2,3], [1,1,1,2,1,3], [1,3,1,2,4], [1,1,1,5], [2,8,3], [3,3,2], [6,1,1], [1,4,2,1,2], [2,4,4], [2,2,1,4], [5,4]
]
col_constraints = [
    [4,9], [4,1,1,3,2], [6,1,1,1,3,2], [2,1,1,1,2,1], [1,1,1,1,1,1,2,1], [3,3,1,4,1], [2,3,1,2,2,2], [1,2,1,3,2], [2,1,1,1,1,2], [5,2,1,1,2], [1,4,2,1], [1,1,2,1,1], [1,1,1,2,1,1], [4,3,3,4], [2,2,6,3], [2,1,1,5,4], [4,5,4]
]
compute_time(row_constraints, col_constraints)

..######.##..####
.###.##..#.#.####
###.##...#...#..#
####..#.##..##.##
###.#.#.##.......
#.#...#.......#..
.#...#.##....###.
..#.####.#..##..#
.#...#..###..####
#.....##..##..###
#.#...#..##.#.###
#..###..#.##.####
#.#....#...#####.
##.########.###..
###..###..##.....
######......#...#
#.####..##...#.##
##.....####..####
.##...##...#.####
..#####.....####.


8.099264144897461

10040 14x14

In [56]:
row_constraints = [
    [4], [8], [10], [12], [12], [3,2,3], [4,2,4], [14], [14], 
    [2,2,2,2], [1,2,2,2,1], [14], [4,4,4], [2,2,2]
]
col_constraints = [
    [7], [7,3], [7,4], [8,3], [4,3,1], [5,3,2], [9,4], [9,4], [5,3,2], [4,3,1], [8,3], [7,4], [7,3], [7]
]
compute_time(row_constraints, col_constraints)

.....####.....
...########...
..##########..
.############.
.############.
.###..##..###.
####..##..####
##############
##############
##..##..##..##
#.##..##..##.#
##############
####.####.####
.##...##...##.


0.16190719604492188

8824 13x13

In [57]:
row_constraints = [
    [5], [7], [2,1,6], [3,1,5], [1,5], [2,5], [1,7], [3,1,5], [1,1,5], [2,4], [7], [2,2], [2,2]
]
col_constraints = [
    [2], [3], [1,1,2], [2,1,1], [2,1,4], [3,2,1], [4,2,3], [7,2], 
    [9,1,1], [12], [10], [7], [4]
]
compute_time(row_constraints, col_constraints)

....#####....
...#######...
##.#.######..
###.#.#####..
.#.....#####.
..##...#####.
....#.#######
..###.#.#####
..#.#...#####
....##...####
.....#######.
......##.##..
.....##.##...


0.08646297454833984

8612 11x11

In [59]:
row_constraints = [
    [5], [9], [2,1,1,1,2], [1,1,1,1,1], [11], [1], [1], [1], [1], [1,1], [3]
]
col_constraints = [
    [3], [2,1], [1, 2], [3,1], [2,1], [11], [2,1,1], [3,1,2], [1,2], [2,1], [3]
]
compute_time(row_constraints, col_constraints)

...#####...
.#########.
##.#.#.#.##
#.#..#..#.#
###########
.....#.....
.....#.....
.....#.....
.....#.....
.....#.#...
.....###...


0.02567291259765625

22501 15x14

In [61]:
row_constraints = [
    [5], [4,2], [1,4,1], [1,4,4], [4,1,1,1], [5,4,1], [1,2,2,2,1], 
    [1,2,3], [4,3], [1,1,3], [1,1,3], [1,1,4], [1,7], [2,1], [5]
]
col_constraints = [
    [5], [1,2,1], [7,3], [7,1,2] ,[4,1,3,1], [1,3,2,1,1], [2,2,1,1], [6,1,1], [1,1,1,1], [1,1,1,1], [1,2,2,1], [1,8], [1,6], [6]
]
compute_time(row_constraints, col_constraints)

..#####.......
.####.##......
#.####.#......
#.####.####...
####.#.#...#..
#####..####.#.
#.##..##..##.#
.#...##....###
..####.....###
..#.#......###
..#.#......###
...#.#....####
...#..#######.
....##.....#..
......#####...


0.2803788185119629

21054 14x12

In [65]:
row_constraints = [
    [1,1,1], [4,1,1], [1,1,1,1,1], [1,1,1,2], [1,1,3,2], [8,1], [1,1,4,1],[4,3], [4,2,1], [2,1,1], [3,1,1], [2,1], [2], [1]
]
col_constraints = [
    [2], [1,1,1,2], [1,1,1,1,2], [2,3,3], [2,1,1,5], [1,5], [1,6,1], [1,6], [1,2,2,1,1], [1,1,1,1,1], [1,1,3], [2,1]
]
compute_time(row_constraints, col_constraints)

....#.#.#...
..####.#.#..
.#.#..#.#.#.
..#.#.#.##..
.#.#.###..##
..########.#
.#.#.####.#.
....####.###
..####.##.#.
...##..#.#..
..###.#.#...
.##.#.......
##..........
#...........


0.12435507774353027

23176 11x20

In [66]:
row_constraints = [
    [1,3], [1,2,1,1], [4,1,3], [1,3,2,1], [6,5], [2,12], [1,1,2,12], [6,10, 1], [17], [1,2,1,2], [1,2,1,2]
]
col_constraints = [
    [1], [1], [2], [3], [1,2], [4,1], [2,4], [1,2,1,1], [10], [8], [9], [1,5], [1,4], [1,1,4,1], [1,5], [1,5,1], [9], [9], [1, 3, 1], [2,2]
]
compute_time(row_constraints, col_constraints)


..........#.###.....
........#.##...#.#..
.......####..#..###.
......#.###.....##.#
......######...#####
....##.############.
#..#.##.############
.######.##########.#
..#################.
......#.##....#.##..
.....#.##....#.##...


4.399397850036621

22867 14x14

In [153]:
row_constraints = [
    [1,6,1], [3,4,3], [3,4,3], [4,4], [1,1,1,1], [4,1,1,4], [4,1,1,4], [4,4], [4,2,2,1], [5,3,1], [4,4,3], [3,2], [3,1,1,1,1], [3,1,1,1,1]
]
col_constraints = [
    [1,10], [3,9], [3,9], [10], [1,1,1,2], [3,2,1], [3,1,1], [3,1,1], [3,2,1], [1,1,1,2], [10], [3,9], [3,3,2], [1,6,2]
]
compute_time(row_constraints, col_constraints)

#...######...#
.###.####.###.
.###.####.###.
.####....####.
#..#......#..#
####.#..#.####
####.#..#.####
####......####
####..##..##.#
#####....###.#
####.####.###.
###........##.
###.#....#.#.#
###.#....#.#.#


0.17143702507019043

25992 15x14

In [69]:
row_constraints = [
    [3], [1,2], [7], [1,2], [2], [2], [2,4], [3,3,1], [6,3], [4,1,2,1], [3,2,1], [2,1,1], [3,2], [3,2], [9]
]
col_constraints = [
    [1,4], [1,6], [4,4,3], [1,1,4,2], [3,5,1], [5,2,1], [2,2,1], [1,1], [2,1], [1,2,1], [1,2,2], [4,2], [1,2], [3]
]
compute_time(row_constraints, col_constraints)

..###.........
..#.##........
#######.......
..#..##.......
....##........
....##........
...##...####..
..###.###..#..
.######....###
####.#....##.#
###......##..#
##.......#..#.
###........##.
.###......##..
..#########...


0.2649810314178467

27390 15x10

In [70]:
row_constraints = [
    [8], [10], [4,5], [1,4], [1,1,3], [1,3], [2,4], [7], [5,1], [1,3], [3,2,3], [2,2,3], [1,2,1,2], [2,1,1], [2,1,2]
]
col_constraints = [
    [2,1,3,1], [4,1,2,2,2], [3,1,3,1,2], [3,4,2], [2,1,2,2,1],
    [3,2,2,1,1], [4,2,1,1], [12], [7,3,1], [5,3,1]
]
compute_time(row_constraints, col_constraints)

.########.
##########
####.#####
.#....####
..#.#..###
.#.....###
..##.####.
.#######..
#####..#..
...#.###..
###.##.###
##.##..###
#.##.#..##
.##.#.#...
##...#..##


0.12304186820983887

28290 20x10

In [73]:
row_constraints = [
    [2,1,2], [1,1,1,1], [1,1], [5], [3,1], [4,2], [3,1], [2,1],
    [5], [3,1] ,[2,1,3], [2,1,3,1], [1,1,2], [7], [2,1], [1,1], [2], [1], [1], [1]
]
col_constraints = [
    [2,8,2], [1,10,2], [1,4,2, 1], [1,3,1,1,1], [1,1,2,2], [1,4,1,2,4], [2,2,1,1,4], [4], [1,1], [2]
]
compute_time(row_constraints, col_constraints)

##.#.##...
#.#.#.#...
.#...#....
.#####....
.###.#....
####.##...
###...#...
##...#....
#####.....
###.#.....
##...#.###
##.#.###.#
#...#..##.
.#######..
##...#....
#....#....
.....##...
......#...
......#...
......#...


4.031236886978149

16. 28999 15x9

In [75]:
row_constraints = [
    [1,1], [1,1], [1,1], [1,1], [1,1], [3,3], [3,3], [4,4], [7], [5], [0], [5], [5], [3], [1]
]
col_constraints = [
    [3], [8], [1,5,2], [3,3], [2,4], [3,3], [1,5,2], [8], [3]
]
compute_time(row_constraints, col_constraints)

..#...#..
.#.....#.
.#.....#.
.#.....#.
.#.....#.
###...###
###...###
####.####
.#######.
..#####..
.........
..#####..
..#####..
...###...
....#....


0.10718894004821777

28860 15x14

In [76]:
row_constraints = [
    [1], [2,1], [4,4], [5,5], [5,2,3], [5,6], [1,3,6], [2,6], [2,6], [12], [11], [9], [10], [9], [10]
]
col_constraints = [
    [5,4], [5,6], [14], [6,6], [4,7], [8], [9], [10], [15], [9,3], [2,3,2,1], [5,1,1], [3], [1]
]
compute_time(row_constraints, col_constraints)

........#.....
.##.....#.....
####....####..
#####...#####.
#####...##.###
#####..######.
#.###.######..
..##.######...
.##.######....
############..
###########...
#########.....
##########....
.#########....
..##########..


0.33782196044921875

18. 28799 11x15

In [77]:
row_constraints = [
    [15], [1,12], [6,3], [2,2,2], [3,1,1,2,1], [1,2,1,1,2], [3,1,1,4], [5,2,5], [1,1,2,1,1], [1,1], [15]
]
col_constraints = [
    [2,1,3], [1,1,1], [1,2,1], [3,4,1], [8,1], [5,2,1], [3,2,1],
    [5,2,1], [4,3,1], [2,2,1], [4,3,1], [8,1], [3,4,1], [2,2,1],
    [2,1,4]
]
compute_time(row_constraints, col_constraints)

###############
#..############
...######.###..
....##.##.##...
...###.#.#.##.#
#..##.#..#.##..
..###.#.#.####.
.#####.##.#####
#....#.##.#...#
#.............#
###############


0.1494889259338379

19. 28688 12x14

In [78]:
row_constraints = [
    [3], [3,1,4], [1,1,1,1,1], [1,1,1,1,3], [1,1,1,1], [7,2,1],
    [1,2,3], [1,1], [1,2], [1,4,3], [1,1,1,1,1], [1,1,1,1,1]
]
col_constraints = [
    [10], [1,1], [1,1,1,2], [2,1,1,1], [1,1], [1,3], [5], [1,1,2], 
    [1,2,1,1], [6,3], [1,3,1], [1,1,1], [3,1], [3]
]
compute_time(row_constraints, col_constraints)

.......###....
.###..#..####.
#..#..#..#..#.
#.#...#..#.###
#.....#..#...#
#######.##...#
#......##.###.
#.........#...
#........##...
#.####.###....
#.#..#.#.#....
#..#.#..#.#...


0.10926103591918945

20. 69300 14x19

In [91]:
row_constraints = [
    [2], [2,3], [2,3],[4], [4,5], [5,8], [2,13], [12,4,1], [11,7], [2,6,7], [6,6], [3,2,4], [3,4], [3,7]
]
col_constraints = [
    [2], [1,4], [3,6] ,[5,2,1], [6,2], [12], [3,8], [2,6], [1,6] ,
    [7,1], [8,1], [4,3], [3,3,2], [10], [9], [7,1], [6], [3], [3]
]
compute_time(row_constraints, col_constraints)

.##................
..##..###..........
..##.###...........
...####............
..####...#####.....
..#####.########...
.##.#############..
############.####.#
###########.#######
.##..######.#######
.....######.######.
.....###..##.####..
....###....####....
...###...#######...


2.7488207817077637

In [151]:
row_constraints = [
    [1,3], [3,1], [1,5,2], [1,7], [10], [6,2], [3,1], [3]
]

col_constraints = [
    [1,1], [1,3], [1,2], [6], [8], [8], [1,4,1], [2], [4], [6]
]

compute_time(row_constraints, col_constraints)


.#..###...
...###...#
#.#####.##
.#.#######
##########
.######.##
...###...#
....###...


0.00432896614074707